## Dataset building

In [ ]:
# imports
import pathlib

import numpy as np
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn_tutorial.datasets import ObjectDetectionDataSet
from pytorch_faster_rcnn_tutorial.transformations import Clip, ComposeDouble
from pytorch_faster_rcnn_tutorial.transformations import FunctionWrapperDouble
from pytorch_faster_rcnn_tutorial.transformations import normalize_01
from pytorch_faster_rcnn_tutorial.utils import get_filenames_of_path
from pytorch_faster_rcnn_tutorial.utils import stats_dataset
from pytorch_faster_rcnn_tutorial.visual import DatasetViewer

In [ ]:
# root directory
root = pathlib.Path('pytorch_faster_rcnn_tutorial/data/heads')

In [ ]:
# input and target files
inputs = get_filenames_of_path(root / 'input')
targets = get_filenames_of_path(root / 'target')

inputs.sort()
targets.sort()

In [ ]:
# mapping
mapping = {
    'head': 1,
}

In [ ]:
# transforms
transforms = ComposeDouble([
    Clip(),
    # AlbumentationWrapper(albumentation=A.HorizontalFlip(p=0.5)),
    # AlbumentationWrapper(albumentation=A.RandomScale(p=0.5, scale_limit=0.5)),
    # AlbumentationWrapper(albumentation=A.VerticalFlip(p=0.5)),
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

In [ ]:
# dataset building
dataset = ObjectDetectionDataSet(inputs=inputs,
                                 targets=targets,
                                 transform=transforms,
                                 use_cache=False,
                                 convert_to_format=None,
                                 mapping=mapping)

In [ ]:
# visualize dataset
color_mapping = {
    1: 'red',
}

datasetviewer = DatasetViewer(dataset, color_mapping)
datasetviewer.napari()
datasetviewer.gui_text_properties(datasetviewer.shape_layer)

In [ ]:
# visualize dataset with Faster-RCNN transformer
color_mapping = {
    1: 'red',
}

transform = GeneralizedRCNNTransform(min_size=1024,
                                     max_size=1024,
                                     image_mean=[0.485, 0.456, 0.406],
                                     image_std=[0.229, 0.224, 0.225])

datasetviewer = DatasetViewer(dataset, color_mapping, rccn_transform=transform)
datasetviewer.napari()

## Dataset statistics

In [ ]:
stats = stats_dataset(dataset)
stats

In [ ]:
transform = GeneralizedRCNNTransform(min_size=1024,
                                     max_size=1024,
                                     image_mean=[0.485, 0.456, 0.406],
                                     image_std=[0.229, 0.224, 0.225])

stats_transform = stats_dataset(dataset, transform)
stats_transform